In [ ]:
import subprocess
import os
import pandas as pd
import numpy as np
import json
import time

from shapely.geometry import Point, Polygon, LineString
from shapely_geojson import *


# Hvor går trafikken?
Denne notebooken henter ned de største domenene fra [Tranco](https://tranco-list.eu/list/MW69/1000000), filtrerer ut alle domener som ikke ender på .no. Da sitter man igjen med en liste over 1343 norske domener. Disse domenene kjøres så gjennom traceroute for å finne ut hvilke IPer trafikken går gjennom på veien. Vi sjekker så hvor serverne til disse IPene befinner seg gjennom GeoIP. 

## Tranco
Listen kan lastes ned her (den ligger også som en .csv i dette repoet): https://tranco-list.eu/list/MW69/1000000

Listen er en sammenstilling av toppdomenene fra Alexa, Umbrella, Majestic og Quantcast fra 2018-12-18 til 2019-01-16 (30 days).

In [ ]:
tranco = pd.read_csv('tranco_MW69.csv', names="domain".split())

In [ ]:
tranco = set(tranco[tranco.domain.str.endswith('.no')].domain.str.replace('www.', ''))

In [ ]:
len(tranco)

## Traceroute-hjelpefunksjoner
Denne notebooken kjøres både av en Windows-maskin og en Mac, så vi har laget en liten funksjon som abstraherer traceroute-kjøringen. Vi har også en liten hjelpefunksjon for å kjøre IPer gjennom Dazzlepod.com for IP-lookups.

In [ ]:
def lookup_site(site_name):
    proc_name = "traceroute"
    if os.name == "nt":
        proc_name = "tracert"
        s = subprocess.Popen([proc_name, '-h', '16', '-w', '1000', site_name], stdout=subprocess.PIPE)
    else: #UNIX
        s = subprocess.Popen([proc_name, '-n', '-w', '0.5', '-q', '1', '-m', '16', site_name], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return s.communicate()

In [ ]:
def get_ip(ip):
    r = requests.get('https://dazzlepod.com/ip/'+ip, headers={'User-Agent': 'curl'}).json()
    return r

## Traceroute-kjøring
**MERK!** Denne kjøringen tar lang tid. Hvis du bare vil leke med kjøringen vi har gjort, gå to bolker ned og kommenter ut `df`-deklarasjonen som laster inn CSV-filen vi har generert. 

In [ ]:
df_nettsider = {}
n = 0

localtime = time.asctime( time.localtime(time.time()) )
print ("Start time:", localtime)

if os.name == "nt": # win - Martin

    print('Using Windows')
    for nettside in tranco:
        hop = 1
        
        print(f"Looking up: {nettside} ({n})")
        n += 1
        data = lookup_site(nettside)
        data = data[0].decode('ascii')
        ips = []
        
#        print(f"data: {data}")

        for i, obj in enumerate(data.split('\r\n')):

#            print(f"obj: {obj}")

            if  obj.find('Tracing route') != -1 or obj.find('net unreachable') != -1 or obj.find('over a') != -1 or obj.find('Unable to resolve') != -1:
#                print("if none")
                None

            elif '*' in obj or len(obj.split('ms ')) < 4:
                #removes shorter lines and unresolved
#                print("elif none")
                None 
            
            elif obj.find('[') != -1: 
#                print(f"obj: {obj}")   
                ips.append((hop, obj.split("[")[1].strip("] ")))
                hop = hop + 1 
#                print(f"'elif append {ips}")
            
            elif len(obj.split('ms ')) == 4:
                ips.append((hop, obj.split(" ms ")[3].strip()))
                hop = hop + 1
#                print(f"'else append {ips}")
        df_nettsider[nettside] = ips
    
else: 
        # Henrik
    print('Using UNIX')
    for nettside in nettsider:
        print("Looking up", nettside)
        data = lookup_site(nettside)
        data = data[0].decode('ascii')
        ips = []
        for i, obj in enumerate(data.split('\n')):
            if '*' in obj or len(obj.split(' ')) < 3:
                break
            ips.append((i+1, obj.split(' ')[3]))

        df_nettsider[nettside] = ips   
        
localtime = time.asctime( time.localtime(time.time()) )
print ("End time:", localtime)

In [ ]:
#creates a geojson and a dataframe
geo_hops = []
features = {}
df_geo = []

tmptf = time.localtime()

for nettside, hops in df_nettsider.items():
    obj = {'website': nettside }
    features[nettside] = []

    for hop, ip in hops:
        got_ip = get_ip(ip)
        lat = 0
        lng = 0
        country = 'NONE'
        if 'longitude' in got_ip and 'latitude' in got_ip and 'country_code' in got_ip:
            lat = got_ip['latitude']
            lng = got_ip['longitude']
            country = got_ip['country_code']

            features[nettside].append(Feature(Point(lng, lat), {'country': country, 'hop': hop, 'website': nettside}))
            df_geo.append([nettside, hop, ip, country, lat, lng])
            
df = pd.DataFrame(df_geo, columns=['website', 'hop', 'ip','country','lat','lgn'])     

df.to_csv('tranco-%d-%02d-%02d.csv' % (tmptf.tm_year, tmptf.tm_mon, tmptf.tm_mday), index=False, encoding='utf-8')

feature_total = []
for nettside, feature_list in features.items():
    feature_total.extend(feature_list)
_featurecol2 = FeatureCollection(feature_total)
with open('tranco-%d-%02d-%02d.geojson' % (tmptf.tm_year, tmptf.tm_mon, tmptf.tm_mday), 'w') as fp:
    fp.write(dumps(_featurecol2, indent=2))

In [ ]:
# Kommentér ut disse for å bruke den pregenererte CSV-filen fra NRKbeta
#df = pd.read_csv('tranco-190124.csv')
#df = tmpdf[tmpdf.website.str.endswith('.no')]
#df.to_csv('tranco-190124.csv', index=False)

In [ ]:
websites = df.groupby(['website', 'country'])

In [ ]:
foreign_hops = len(websites.filter(lambda x: 'NO' not in x.country.unique()).website.unique())

In [ ]:
print("Antall nettsider hvor minst ett hopp går utenfor Norge: %d" % foreign_hops)
print("Prosentandel av nettsider hvor minst ett hopp går utenfor Norge: %f" % (foreign_hops/len(df.website.unique())*100))